# Spectractor Simulation

Goal is to process a series of images in order to produce spectra

- author Sylvie Dagoret-Campagne
- date : April 05th 2018
- update : April 09th 2018


In [1]:
theNotebook="Run_SpectractorSim"

# Spectractor Launcher

In [2]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

## Required packages 

In [3]:
import matplotlib.pyplot as plt
import sys
import os
import pandas as pd

In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10, 6),   #defines the default image size
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [6]:
plt.rcParams.update({'figure.max_open_warning': 0}) # to see a large number of files

## Tools

Spectractor is supposed to be installed in paralel to Spectractorsim

In [7]:
PATH_SPECTRACTOR='../../Spectractor'
PATH_SPECTRACTORSIM='../../SpectractorSim'
PATH_SPECTRACTORANAATM='../../SpectractorAnaAtm'

In [8]:
PATH_SPECTRACTOR

'../../Spectractor'

In [9]:
sys.path.append(PATH_SPECTRACTOR)
sys.path.append(PATH_SPECTRACTORSIM)

## Spectractor 

In [10]:
from spectractor import *

In [11]:
from tools import *
from holo_specs import *
from targets import *
from optics import *
import parameters 

## SpectractorSim

In [12]:
from spectractorsim import *

# Configuration

## Input Directory

### So use HD205905

In [13]:
if 0:
    #inputdir="/sps/lsst/data/AtmosphericCalibration/CTIODataJune2017_reduced_RG715_v2/data_30may17"
    inputdir="/Users/dagoret/DATA/CTIODataJune2017_reduced_RG715_v2/data_30may17"
    #MIN_IMGNUMBER=208
    MIN_IMGNUMBER=230
    MAX_IMGNUMBER=284
    date="20170530"
    object_name='HD205905'
    tag = "reduc"
    SelectTagRe='^%s_%s_([0-9]+).fits$' % (tag,date) # regular expression to select the file
    SearchTagRe='^%s_%s_[0-9]+.fits$' % (tag,date)
    NBIMGPERROW=2
    subdir=os.path.basename(inputdir)

In [14]:
# Simulation April 9th 2018 by Sylvie
#inputdir="/sps/lsst/data/AtmosphericCalibration/CTIODataJune2017_reduced_RG715_v2/data_30may17"
inputdir="/Users/dagoret/DATA/CTIODataJune2017_reduced_RG715_v2/data_28may17"
#MIN_IMGNUMBER=208
MIN_IMGNUMBER=56
MAX_IMGNUMBER=135
date="20170528"
object_name='HD185975'
tag = "reduc"
SelectTagRe='^%s_%s_([0-9]+).fits$' % (tag,date) # regular expression to select the file
SearchTagRe='^%s_%s_[0-9]+.fits$' % (tag,date)
NBIMGPERROW=2
subdir=os.path.basename(inputdir)

# Output directory

In [15]:
dir_top_images="images_notebooks/"+theNotebook ### Top directory to save images
ensure_dir(dir_top_images) # create that directory
outputdir="./simspectra"
outputdir=os.path.join(outputdir,subdir)
#ensure_dir(outputdir)

# Logbook

For the moment, the logbook is in the local directory

In [16]:
ls ../../SpectractorAnaAtm/common_info

ctiofulllogbook_jun2017_v3.csv   ctiofulllogbook_jun2017_v4.xlsx
ctiofulllogbook_jun2017_v3.xlsx


In [17]:
file_logbook_csv='ctiofulllogbook_jun2017_v3.csv'
file_logbook_xlsx='../ctiofulllogbook_jun2017_v3.xlsx'
file_logbook_xlsx=os.path.join(PATH_SPECTRACTORANAATM,'common_info/ctiofulllogbook_jun2017_v4.xlsx')

In [18]:
file_logbook_xlsx

'../../SpectractorAnaAtm/common_info/ctiofulllogbook_jun2017_v4.xlsx'

In [19]:
#df=pd.read_csv(file_logbook_csv,index_col=0)
df=pd.read_excel(file_logbook_xlsx,index_col=0)

In [20]:
df.head()

,date,subdir,index,object,filter,disperser,airmass,exposure,focus,seeing,...,T,RH,W,file,Obj-posXpix,Obj-posYpix,Dx,Dy,comment,skip
31,2017-06-02T04:26:04.602,data_01jun17,15.0,HD185975,dia,HoloPhAg,1.919,120.0,12400.0,0.759,...,12.8,22.0,21.3,reduc_20170601_015.fits,NaN,NaN,NaN,NaN,mesure holo,NaN
30,2017-06-02T04:29:01.302,data_01jun17,16.0,HD185975,dia,HoloPhAg,1.918,120.0,12400.0,0.759,...,12.9,22.0,12.1,reduc_20170601_016.fits,NaN,NaN,NaN,NaN,mesure holo,NaN
99,2017-06-02T04:31:58.001,data_01jun17,17.0,HD185975,dia,HoloPhAg,1.917,120.0,12400.0,0.745,...,13.0,22.0,14.3,reduc_20170601_017.fits,NaN,NaN,NaN,NaN,mesure holo,NaN
50,2017-06-02T04:34:52.548,data_01jun17,18.0,HD185975,dia,HoloPhAg,1.917,120.0,12400.0,0.742,...,13.2,21.0,20.2,reduc_20170601_018.fits,NaN,NaN,NaN,NaN,mesure holo,NaN
6,2017-06-02T04:37:47.745,data_01jun17,19.0,HD185975,dia,HoloPhAg,1.916,120.0,12400.0,0.728,...,13.2,21.0,24.4,reduc_20170601_019.fits,NaN,NaN,NaN,NaN,mesure holo,NaN


# Extract the relevant data and sort by the column index

In [21]:
all_obs=df.loc[(df["subdir"]==subdir) & (df["object"]==object_name),:].sort_values(by=['index'])

In [22]:
all_obs=all_obs.loc[(all_obs["index"]>=MIN_IMGNUMBER) &  (all_obs["index"]<=MAX_IMGNUMBER),:]

In [23]:
all_obs.head()

,date,subdir,index,object,filter,disperser,airmass,exposure,focus,seeing,...,T,RH,W,file,Obj-posXpix,Obj-posYpix,Dx,Dy,comment,skip
78,2017-05-29T04:16:11.205,data_28may17,56.0,HD185975,dia,HoloAmAg,1.931,30.0,12550.0,1.45,...,6.8,38.0,35.6,reduc_20170528_056.fits,780.0,650.0,50.0,100.0,atmosphère,NaN
63,2017-05-29T04:18:29.109,data_28may17,57.0,HD185975,dia,Thor300,1.929,30.0,12550.0,1.45,...,6.8,38.0,33.6,reduc_20170528_057.fits,780.0,650.0,50.0,100.0,atmosphère,NaN
7,2017-05-29T04:19:51.453,data_28may17,58.0,HD185975,dia,HoloPhP,1.929,30.0,12550.0,1.45,...,6.9,38.0,28.1,reduc_20170528_058.fits,780.0,650.0,50.0,100.0,atmosphère,NaN
73,2017-05-29T04:21:13.848,data_28may17,59.0,HD185975,dia,HoloPhAg,1.928,30.0,12550.0,1.45,...,6.6,39.0,24.8,reduc_20170528_059.fits,780.0,650.0,50.0,100.0,atmosphère,NaN
98,2017-05-29T04:27:42.796,data_28may17,60.0,HD185975,dia,Thor300,1.924,30.0,12550.0,1.45,...,6.2,41.0,29.2,reduc_20170528_060.fits,780.0,650.0,50.0,100.0,atmosphère,NaN


# Extract relevant info for SpectractorSim

In [24]:
target_tosimspec=all_obs[['index','object','airmass','filter','disperser','exposure','P','T','RH','file','date']]

In [25]:
target_tosimspec.head()

,index,object,airmass,filter,disperser,exposure,P,T,RH,file,date
78,56.0,HD185975,1.931,dia,HoloAmAg,30.0,777.0,6.8,38.0,reduc_20170528_056.fits,2017-05-29T04:16:11.205
63,57.0,HD185975,1.929,dia,Thor300,30.0,777.0,6.8,38.0,reduc_20170528_057.fits,2017-05-29T04:18:29.109
7,58.0,HD185975,1.929,dia,HoloPhP,30.0,778.0,6.9,38.0,reduc_20170528_058.fits,2017-05-29T04:19:51.453
73,59.0,HD185975,1.928,dia,HoloPhAg,30.0,778.0,6.6,39.0,reduc_20170528_059.fits,2017-05-29T04:21:13.848
98,60.0,HD185975,1.924,dia,Thor300,30.0,778.0,6.2,41.0,reduc_20170528_060.fits,2017-05-29T04:27:42.796


### Setting the parameters of SpectractorSim

In [ ]:
parameters.DEBUG = False
parameters.VERBOSE = False

In [ ]:
for index, row in target_tosimspec.iterrows():
    index=row['file']
    num=row['index']
    targetname=row['object']
    airmass=row['airmass']
    filename_data=row['file']
    filtername=row['filter']
    dispersername=row['disperser']
    exposure=row['exposure']
    pressure=row['P']
    temperature=row['T']
    rhumidity=row['RH']
    date=row['date']
    
    print '*************************************************************************************************'
    print '*********** spectrum id=',index, ' num =',num,' airmass =',airmass,' pressure =',pressure, '*****'
    print '*************************************************************************************************'
    
   
    SpectractorSim(filename_data,outputdir,targetname,index,airmass,pressure,temperature,rhumidity,exposure,filtername,dispersername)
    

*************************************************************************************************
*********** spectrum id= reduc_20170528_056.fits  num = 56.0  airmass = 1.931  pressure = 777.0 *****
*************************************************************************************************
Loading /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/calspec/hd185975_stis_001.fits
Loading /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/calspec/hd185975_stis_002.fits
Loading /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/calspec/hd185975_stis_003.fits
*************************************************************************************************
*********** spectrum id= reduc_20170528_057.fits  num = 57.0  airmass = 1.929  pressure = 777.0 *****
*************************************************************************************************
Loading /Users/dagoret/MacOSX/External/PySynPhotData/grp/hst/cdbs/calspec/hd185975_stis_001.fits
Loading /Users/d